In [ ]:
!pip install torch optuna scikit-learn pandas -q

import os
import shutil
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.preprocessing import StandardScaler
import optuna
from google.colab import files

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 593.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 37.2 MB/s eta 0:00:00


In [ ]:
"""
Preprocess The data
"""
def preprocess_data(file_path):
    raw_data = pd.read_csv(file_path)
    raw_data.drop(index=0, inplace=True)  # Remove the first row which is the indexes
    X = raw_data.iloc[:, :-1].values  # First 16 columns as features
    y = raw_data.iloc[:, -1].values   # 17th column as labels
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.long)

"""
Define the neural network architecture
"""
class SmartMattressNet(nn.Module):
    def __init__(self, units_1, units_2, units_3, dropout_1, dropout_2, dropout_3):
        super(SmartMattressNet, self).__init__()
        self.fc1 = nn.Linear(16, units_1)
        self.bn1 = nn.BatchNorm1d(units_1)
        self.dropout1 = nn.Dropout(dropout_1)

        self.fc2 = nn.Linear(units_1, units_2)
        self.bn2 = nn.BatchNorm1d(units_2)
        self.dropout2 = nn.Dropout(dropout_2)

        self.fc3 = nn.Linear(units_2, units_3)
        self.bn3 = nn.BatchNorm1d(units_3)
        self.dropout3 = nn.Dropout(dropout_3)

        self.fc4 = nn.Linear(units_3, 6)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout3(x)
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

"""
Custom loss function
"""
def custom_loss(output, target):
    loss = F.cross_entropy(output, target, reduction='none')
    # Increase penalty for stomach vs. back misclassification
    stomach_back_mask = (target == 1) & (output.argmax(dim=1) == 0)
    loss = torch.where(stomach_back_mask, loss * 2, loss)
    return loss.mean()  # Take the mean of the loss

"""
The Hyperparameter optimization setup and run
"""
def objective(trial):
    units_1 = trial.suggest_int('units_1', 32, 128, step=16)
    units_2 = trial.suggest_int('units_2', 16, 64, step=16)
    units_3 = trial.suggest_int('units_3', 8, 32, step=8)
    dropout_1 = trial.suggest_float('dropout_1', 0.2, 0.5, step=0.1)
    dropout_2 = trial.suggest_float('dropout_2', 0.2, 0.5, step=0.1)
    dropout_3 = trial.suggest_float('dropout_3', 0.2, 0.5, step=0.1)
    learning_rate = trial.suggest_categorical('learning_rate', [1e-2, 1e-3, 1e-4])

    model = SmartMattressNet(units_1, units_2, units_3, dropout_1, dropout_2, dropout_3).to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=32, shuffle=False)

    for epoch in range(50):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            output = model(batch_X.to(device))
            loss = custom_loss(output, batch_y.to(device))
            loss.backward()
            optimizer.step()

        model.eval()
        val_loss = 0
        correct = 0
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                output = model(batch_X.to(device))
                val_loss += custom_loss(output, batch_y.to(device)).item()
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(batch_y.view_as(pred).to(device)).sum().item()

        val_loss /= len(val_loader.dataset)
        accuracy = correct / len(val_loader.dataset)

        trial.report(accuracy, epoch)

        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy

"""
evaluating the model
"""
def evaluate_model(model, X, y):
    model.eval()
    with torch.no_grad():
        output = model(X.to(device))
    predictions = output.argmax(dim=1)
    accuracy = (predictions == y.to(device)).float().mean().item()
    return accuracy

"""
The main function to load the data, create the model, and train the model
"""
def main():
    print("Upload the training data CSV file:")
    train_data_file = files.upload()
    train_file_path = list(train_data_file.keys())[0]
    global X_train, y_train
    X_train, y_train = preprocess_data(train_file_path)

    print("Upload the validation data CSV file:")
    val_data_file = files.upload()
    val_file_path = list(val_data_file.keys())[0]
    global X_val, y_val
    X_val, y_val = preprocess_data(val_file_path)

    print("Upload the test data CSV file:")
    test_data_file = files.upload()
    test_file_path = list(test_data_file.keys())[0]
    global X_test, y_test
    X_test, y_test = preprocess_data(test_file_path)

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=10)

    best_trial = study.best_trial
    best_params = best_trial.params
    print(f"Best trial parameters: {best_params}")

    # Extract model parameters from best_params
    model_params = {k: best_params[k] for k in ['units_1', 'units_2', 'units_3', 'dropout_1', 'dropout_2', 'dropout_3']}
    learning_rate = best_params['learning_rate']

    best_model = SmartMattressNet(**model_params).to(device)
    optimizer = optim.Adam(best_model.parameters(), lr=learning_rate)

    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=32, shuffle=False)

    for epoch in range(50):
        best_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            output = best_model(batch_X.to(device))
            loss = custom_loss(output, batch_y.to(device))
            loss.backward()
            optimizer.step()

    accuracy = evaluate_model(best_model, X_test, y_test)
    print(f"Model evaluation accuracy: {accuracy * 100:.2f}%")
    torch.save(best_model.state_dict(), 'smart_mattress_model.pth')
    files.download('smart_mattress_model.pth')
    print("Model training complete and saved as 'smart_mattress_model.pth'.")

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    main()

Upload the training data CSV file:


Saving merged_data.csv to merged_data.csv
Upload the internal validation data CSV file:


Saving merged_data.csv to merged_data (1).csv
Upload the accuracy test data CSV file:


[I 2024-07-30 08:46:42,467] A new study created in memory with name: no-name-56aefde8-cd97-47d5-aab9-bdc9a67ebf5f


Saving merged_data.csv to merged_data (2).csv


[I 2024-07-30 08:46:56,699] Trial 0 finished with value: 0.8751926040061633 and parameters: {'units_1': 48, 'units_2': 32, 'units_3': 16, 'dropout_1': 0.5, 'dropout_2': 0.30000000000000004, 'dropout_3': 0.5, 'learning_rate': 0.0001}. Best is trial 0 with value: 0.8751926040061633.
[I 2024-07-30 08:47:09,346] Trial 1 finished with value: 0.9707241910631741 and parameters: {'units_1': 80, 'units_2': 64, 'units_3': 32, 'dropout_1': 0.30000000000000004, 'dropout_2': 0.5, 'dropout_3': 0.5, 'learning_rate': 0.001}. Best is trial 1 with value: 0.9707241910631741.
[I 2024-07-30 08:47:21,478] Trial 2 finished with value: 0.9784283513097073 and parameters: {'units_1': 80, 'units_2': 64, 'units_3': 32, 'dropout_1': 0.30000000000000004, 'dropout_2': 0.5, 'dropout_3': 0.2, 'learning_rate': 0.0001}. Best is trial 2 with value: 0.9784283513097073.
[I 2024-07-30 08:47:33,439] Trial 3 finished with value: 0.9506933744221879 and parameters: {'units_1': 48, 'units_2': 48, 'units_3': 16, 'dropout_1': 0.30

Best trial parameters: {'units_1': 96, 'units_2': 48, 'units_3': 32, 'dropout_1': 0.5, 'dropout_2': 0.2, 'dropout_3': 0.30000000000000004, 'learning_rate': 0.01}
Model evaluation accuracy: 98.31%


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Model training complete and saved as 'smart_mattress_model.pth'.
